# Task 13.2: Building Custom Tools for AI Agents

**Module:** 13 - AI Agents & Agentic Systems  
**Time:** 3 hours  
**Difficulty:** ⭐⭐⭐ (Intermediate)

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- [ ] Understand what tools are and why agents need them
- [ ] Create 5 production-ready custom tools
- [ ] Implement safe code execution tools
- [ ] Build agents that use tools to solve problems
- [ ] Handle tool errors gracefully

---

## 📚 Prerequisites

- Completed: Task 13.1 (RAG Pipeline)
- Knowledge of: Python functions, decorators, error handling

---

## 🌍 Real-World Context

**The Problem:** LLMs can only generate text - they can't actually DO things.

**Examples:**
- 🔢 "Calculate 15% tip on $127.50" → LLMs often make arithmetic errors
- 🌐 "What's the weather in Tokyo?" → LLMs don't have real-time data
- 📁 "Read my config.json file" → LLMs can't access your filesystem
- 💻 "Run this Python code" → LLMs can't execute code

**Tools solve this** by giving agents the ability to perform actions and get real results!

---

## 🧒 ELI5: What Are Agent Tools?

> **Imagine a super-smart friend who knows everything BUT has no hands...** 🤔
>
> They can tell you HOW to bake a cake, but can't actually mix the ingredients.
> They can explain HOW to fix a car, but can't hold a wrench.
>
> **Tools are like giving that friend hands!**
>
> - "I need to calculate 15% of 127.50" → 🧮 Uses calculator tool → Gets exact answer
> - "What's in that file?" → 📄 Uses file reader tool → Sees actual contents
> - "Search for latest news" → 🔍 Uses web search tool → Gets real results
>
> **In AI terms:**
> - The LLM decides WHAT tool to use and WITH what inputs
> - The tool EXECUTES the action and returns results
> - The LLM interprets the results and continues reasoning

```
┌─────────────────────────────────────────────────────────────┐
│                    Agent Tool Loop                          │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  User: "What's 15% tip on $127.50 bill?"                   │
│              │                                              │
│              ▼                                              │
│     ┌────────────────┐                                      │
│     │  LLM Thinks:   │                                      │
│     │ "I need to     │                                      │
│     │  calculate..." │                                      │
│     └───────┬────────┘                                      │
│             │                                               │
│             ▼                                               │
│  ┌──────────────────────────────────┐                       │
│  │ Tool Selection:                  │                       │
│  │   🧮 Calculator                  │ ← Decides which tool  │
│  │   Input: "127.50 * 0.15"         │ ← Formats input       │
│  └──────────────┬───────────────────┘                       │
│                 │                                           │
│                 ▼                                           │
│  ┌──────────────────────────────────┐                       │
│  │ Tool Execution:                  │                       │
│  │   Result: 19.125                 │ ← Real calculation!   │
│  └──────────────┬───────────────────┘                       │
│                 │                                           │
│                 ▼                                           │
│     ┌────────────────┐                                      │
│     │ LLM Response:  │                                      │
│     │ "The 15% tip   │                                      │
│     │ is $19.13"     │                                      │
│     └────────────────┘                                      │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

---

## Part 1: Environment Setup

In [ ]:
# Standard imports
import os
import sys
from pathlib import Path
from typing import Optional, Dict, Any, List
import json
import time

# Add scripts to path
sys.path.insert(0, str(Path.cwd().parent / 'scripts'))

# LangChain imports
from langchain.tools import tool, Tool, StructuredTool
from langchain.agents import AgentExecutor, create_react_agent
from langchain.prompts import PromptTemplate
from langchain_community.llms import Ollama
from langchain.callbacks import StdOutCallbackHandler

print("Imports successful!")

In [ ]:
# Initialize the LLM
llm = Ollama(
    model="llama3.1:8b",
    request_timeout=120.0,  # 2 minute timeout for long generations
    temperature=0.1,  # Low temperature for more consistent tool use
    base_url="http://localhost:11434"
)

print("LLM initialized!")

---

## Part 2: Your First Tool - The Calculator

Let's start simple: a calculator that can evaluate mathematical expressions safely.

In [ ]:
# TOOL 1: Safe Calculator
import ast
import operator
import math

class SafeCalculator:
    """
    A safe mathematical expression evaluator.
    
    Unlike using eval(), this only allows mathematical operations,
    making it safe from code injection attacks.
    """
    
    OPERATORS = {
        ast.Add: operator.add,
        ast.Sub: operator.sub,
        ast.Mult: operator.mul,
        ast.Div: operator.truediv,
        ast.FloorDiv: operator.floordiv,
        ast.Mod: operator.mod,
        ast.Pow: operator.pow,
        ast.USub: operator.neg,
    }
    
    FUNCTIONS = {
        'sin': math.sin,
        'cos': math.cos,
        'tan': math.tan,
        'sqrt': math.sqrt,
        'log': math.log,
        'log10': math.log10,
        'exp': math.exp,
        'abs': abs,
        'round': round,
        'floor': math.floor,
        'ceil': math.ceil,
    }
    
    CONSTANTS = {
        'pi': math.pi,
        'e': math.e,
    }
    
    def evaluate(self, expression: str) -> float:
        """Safely evaluate a mathematical expression."""
        try:
            tree = ast.parse(expression, mode='eval')
            return self._eval_node(tree.body)
        except Exception as e:
            raise ValueError(f"Invalid expression: {e}")
    
    def _eval_node(self, node) -> float:
        if isinstance(node, ast.Constant):
            return node.value
        
        elif isinstance(node, ast.Name):
            if node.id in self.CONSTANTS:
                return self.CONSTANTS[node.id]
            raise ValueError(f"Unknown constant: {node.id}")
        
        elif isinstance(node, ast.BinOp):
            left = self._eval_node(node.left)
            right = self._eval_node(node.right)
            op_type = type(node.op)
            if op_type not in self.OPERATORS:
                raise ValueError(f"Unsupported operator")
            return self.OPERATORS[op_type](left, right)
        
        elif isinstance(node, ast.UnaryOp):
            operand = self._eval_node(node.operand)
            if isinstance(node.op, ast.USub):
                return -operand
            raise ValueError("Unsupported unary operator")
        
        elif isinstance(node, ast.Call):
            func_name = node.func.id if isinstance(node.func, ast.Name) else None
            if func_name not in self.FUNCTIONS:
                raise ValueError(f"Unknown function: {func_name}")
            args = [self._eval_node(arg) for arg in node.args]
            return self.FUNCTIONS[func_name](*args)
        
        else:
            raise ValueError(f"Unsupported expression")

In [ ]:
# Create the calculator tool using the @tool decorator
@tool
def calculate(expression: str) -> str:
    """
    Safely evaluate a mathematical expression.
    
    Use this tool when you need to perform calculations. Supports:
    - Basic arithmetic: +, -, *, /, //, %, **
    - Functions: sin, cos, tan, sqrt, log, exp, abs, round, floor, ceil
    - Constants: pi, e
    
    Args:
        expression: A mathematical expression like "sqrt(16) + 2**3"
    
    Returns:
        The result as a string
    """
    try:
        calc = SafeCalculator()
        result = calc.evaluate(expression)
        # Format nicely
        if isinstance(result, float) and result.is_integer():
            return f"{int(result)}"
        return f"{result:.10g}"
    except Exception as e:
        return f"Error: {str(e)}"

# Test the calculator
print("Testing Calculator Tool:")
print(f"  2 + 2 = {calculate.invoke('2 + 2')}")
print(f"  127.50 * 0.15 = {calculate.invoke('127.50 * 0.15')}")
print(f"  sqrt(16) + 2**3 = {calculate.invoke('sqrt(16) + 2**3')}")
print(f"  sin(pi/2) = {calculate.invoke('sin(pi/2)')}")

### 🔍 What Just Happened?

We created a **safe** calculator that:
1. Uses Python's AST (Abstract Syntax Tree) to parse expressions
2. Only allows whitelisted operations and functions
3. Prevents code injection (unlike `eval()`!)
4. Returns helpful error messages

**Security Note:** Never use `eval()` in production code! It can execute arbitrary Python code.

---

## Part 3: File Operations Tool

Let's create a tool that can read files safely.

In [ ]:
# TOOL 2: File Reader
@tool
def read_file(file_path: str) -> str:
    """
    Read the contents of a text file.
    
    Use this tool when you need to read a file from the filesystem.
    
    Args:
        file_path: Path to the file to read
    
    Returns:
        The file contents or an error message
    """
    try:
        path = Path(file_path)
        
        # Security checks
        if not path.exists():
            return f"Error: File not found: {file_path}"
        
        if not path.is_file():
            return f"Error: Not a file: {file_path}"
        
        # Check file size (limit to 100KB)
        file_size = path.stat().st_size
        if file_size > 100_000:
            return f"Error: File too large ({file_size} bytes). Maximum is 100KB."
        
        # Only allow text files
        allowed_extensions = {'.txt', '.md', '.json', '.py', '.csv', '.yaml', '.yml'}
        if path.suffix.lower() not in allowed_extensions:
            return f"Error: Unsupported file type: {path.suffix}"
        
        # Read file
        with open(path, 'r', encoding='utf-8') as f:
            content = f.read()
        
        return content
        
    except UnicodeDecodeError:
        return "Error: File is not a text file or uses unsupported encoding"
    except PermissionError:
        return f"Error: Permission denied for file: {file_path}"
    except Exception as e:
        return f"Error reading file: {str(e)}"

# Test the file reader
data_dir = Path.cwd().parent / "data"
test_file = data_dir / "README.md"

print("Testing File Reader Tool:")
result = read_file.invoke(str(test_file))
print(f"Reading {test_file.name}:")
print(result[:500] + "..." if len(result) > 500 else result)

---

## Part 4: Web Search Tool (Simulated)

In production, you'd connect to a real search API. Here's a simulated version for learning.

In [ ]:
# TOOL 3: Web Search (Simulated)
@tool
def web_search(query: str) -> str:
    """
    Search the web for information.
    
    Use this tool when you need to find current information from the internet.
    
    Args:
        query: The search query
    
    Returns:
        Search results as formatted text
    """
    # Simulated knowledge base for demo
    knowledge = {
        "dgx spark memory": "The NVIDIA DGX Spark has 128GB of unified LPDDR5X memory.",
        "dgx spark gpu": "DGX Spark features the NVIDIA Blackwell GB10 Superchip with 6,144 CUDA cores.",
        "langchain": "LangChain is a framework for developing LLM-powered applications. Latest version is 0.1.x.",
        "llama 3": "Llama 3.1 is Meta's latest open-source LLM, available in 8B, 70B, and 405B parameter versions.",
        "python version": "Python 3.12 is the latest stable version, released in October 2023.",
    }
    
    query_lower = query.lower()
    
    # Find matching results
    results = []
    for key, value in knowledge.items():
        if any(word in query_lower for word in key.split()):
            results.append(value)
    
    if results:
        return "Search Results:\n" + "\n".join(f"- {r}" for r in results)
    else:
        return f"No results found for: {query}. Try a different search query."

# Test the search
print("Testing Web Search Tool:")
print(web_search.invoke("DGX Spark GPU specs"))

---

## Part 5: Code Execution Tool (Sandboxed)

⚠️ **Security Warning:** Running arbitrary code is dangerous! This is a heavily restricted version.

In [ ]:
# TOOL 4: Safe Python Executor
import io
import contextlib
import re

@tool
def execute_python(code: str) -> str:
    """
    Execute Python code in a restricted environment.
    
    Use this when you need to run Python code to solve a problem.
    Available: math, random, json, re, datetime.
    NOT available: file operations, network, system commands.
    
    Args:
        code: Python code to execute
    
    Returns:
        The output of the code execution
    """
    # Security: Block dangerous patterns
    forbidden_patterns = [
        r'\bimport\s+os\b', r'\bimport\s+sys\b', r'\bimport\s+subprocess\b',
        r'\bimport\s+socket\b', r'\bimport\s+requests\b', r'\bimport\s+urllib\b',
        r'\bopen\s*\(', r'\bexec\s*\(', r'\beval\s*\(',
        r'__builtins__', r'__import__',
    ]
    
    for pattern in forbidden_patterns:
        if re.search(pattern, code):
            return "Error: Code contains forbidden operations (file access, network, etc.)"
    
    # Create restricted globals
    import math
    import random
    import json as json_module
    import re as re_module
    from datetime import datetime, timedelta
    
    restricted_globals = {
        '__builtins__': {
            'print': print, 'len': len, 'range': range, 'enumerate': enumerate,
            'zip': zip, 'map': map, 'filter': filter, 'sum': sum, 'min': min,
            'max': max, 'abs': abs, 'round': round, 'sorted': sorted,
            'reversed': reversed, 'list': list, 'dict': dict, 'set': set,
            'tuple': tuple, 'str': str, 'int': int, 'float': float,
            'bool': bool, 'type': type, 'isinstance': isinstance,
            'True': True, 'False': False, 'None': None,
        },
        'math': math,
        'random': random,
        'json': json_module,
        're': re_module,
        'datetime': datetime,
        'timedelta': timedelta,
    }
    
    try:
        # Capture output
        output_buffer = io.StringIO()
        with contextlib.redirect_stdout(output_buffer):
            exec(code, restricted_globals)
        
        output = output_buffer.getvalue().strip()
        return output if output else "Code executed successfully (no output)"
        
    except Exception as e:
        return f"Error: {type(e).__name__}: {str(e)}"

# Test the code executor
print("Testing Code Executor Tool:")
print("\nTest 1: Simple calculation")
print(execute_python.invoke("print(sum([1, 2, 3, 4, 5]))"))

print("\nTest 2: Using math module")
print(execute_python.invoke("import math; print(f'Pi = {math.pi:.4f}')"))

print("\nTest 3: Forbidden operation (should fail)")
print(execute_python.invoke("import os; os.system('ls')"))

---

## Part 6: API Calling Tool

A tool for making HTTP requests to external APIs.

In [ ]:
# TOOL 5: API Caller
import urllib.request
import urllib.parse
import urllib.error

@tool
def call_api(url: str) -> str:
    """
    Make a GET request to an API endpoint.
    
    Use this to fetch data from public APIs.
    
    Args:
        url: The API endpoint URL (must be HTTPS)
    
    Returns:
        The API response or an error message
    """
    try:
        # Security: Validate URL
        parsed = urllib.parse.urlparse(url)
        
        if parsed.scheme not in ('http', 'https'):
            return "Error: Only HTTP/HTTPS URLs are allowed"
        
        # Block internal/local URLs
        blocked_hosts = ['localhost', '127.0.0.1', '0.0.0.0', '169.254.169.254']
        if parsed.hostname in blocked_hosts:
            return f"Error: Access to {parsed.hostname} is not allowed"
        
        # Make request
        headers = {
            'User-Agent': 'DGX-Spark-Agent/1.0',
            'Accept': 'application/json'
        }
        req = urllib.request.Request(url, headers=headers)
        
        with urllib.request.urlopen(req, timeout=10) as response:
            content = response.read().decode('utf-8')
            
            # Truncate if too long
            if len(content) > 2000:
                content = content[:2000] + "\n... (truncated)"
            
            return f"Status: {response.status}\n\n{content}"
            
    except urllib.error.HTTPError as e:
        return f"HTTP Error {e.code}: {e.reason}"
    except urllib.error.URLError as e:
        return f"URL Error: {str(e.reason)}"
    except Exception as e:
        return f"Error: {str(e)}"

# Test the API caller
print("Testing API Caller Tool:")
print("\nFetching from a public API:")
# Using a simple public API for testing
result = call_api.invoke("https://httpbin.org/get")
print(result[:500] + "..." if len(result) > 500 else result)

---

## Part 7: Building an Agent with Tools

Now let's combine all our tools into a working agent!

In [ ]:
# Collect all tools
tools = [
    calculate,
    read_file,
    web_search,
    execute_python,
    call_api,
]

print("Available Tools:")
for tool in tools:
    print(f"  - {tool.name}: {tool.description.split(chr(10))[0][:60]}...")

In [ ]:
# Create a ReAct agent prompt
REACT_PROMPT = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate.from_template(REACT_PROMPT)

print("ReAct prompt template created!")

In [ ]:
# Create the agent
agent = create_react_agent(llm, tools, prompt)

# Create the agent executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,  # Show agent's thinking
    max_iterations=5,
    handle_parsing_errors=True,
)

print("Agent created successfully!")

In [ ]:
# Test the agent with a calculation task
print("="*60)
print("TEST 1: Calculation")
print("="*60 + "\n")

result = agent_executor.invoke({
    "input": "What is 15% tip on a $127.50 restaurant bill?"
})

print("\nFinal Answer:", result['output'])

In [ ]:
# Test the agent with a search task
print("="*60)
print("TEST 2: Information Search")
print("="*60 + "\n")

result = agent_executor.invoke({
    "input": "What is the memory capacity of DGX Spark?"
})

print("\nFinal Answer:", result['output'])

In [ ]:
# Test the agent with a code execution task
print("="*60)
print("TEST 3: Code Execution")
print("="*60 + "\n")

result = agent_executor.invoke({
    "input": "Write and run Python code to calculate the first 10 Fibonacci numbers"
})

print("\nFinal Answer:", result['output'])

---

## ⚠️ Common Mistakes

### Mistake 1: Missing Tool Descriptions

In [ ]:
# ❌ Wrong: No description - agent doesn't know when to use it
# @tool
# def bad_tool(x):
#     return x * 2

# ✅ Right: Clear description with examples
@tool
def double_number(number: str) -> str:
    """
    Double a number.
    
    Use this when you need to multiply a number by 2.
    
    Args:
        number: The number to double (as a string)
    
    Returns:
        The doubled number
    
    Example:
        double_number("5") -> "10"
    """
    return str(float(number) * 2)

print("Good tool descriptions help the agent know when and how to use the tool.")

### Mistake 2: Not Handling Errors

In [ ]:
# ❌ Wrong: Crashes on invalid input
# @tool
# def bad_divide(expression: str) -> str:
#     a, b = expression.split("/")
#     return str(int(a) / int(b))  # Crashes on "10/0"

# ✅ Right: Handle errors gracefully
@tool
def safe_divide(expression: str) -> str:
    """Divide two numbers. Format: 'a/b'"""
    try:
        parts = expression.split("/")
        if len(parts) != 2:
            return "Error: Use format 'a/b' like '10/2'"
        a, b = float(parts[0].strip()), float(parts[1].strip())
        if b == 0:
            return "Error: Cannot divide by zero"
        return str(a / b)
    except ValueError:
        return "Error: Both values must be numbers"

print(safe_divide.invoke("10/0"))  # Gracefully handles error

### Mistake 3: Allowing Unsafe Operations

In [ ]:
# ❌ DANGEROUS: Never do this!
# @tool
# def run_code(code: str) -> str:
#     return str(eval(code))  # Can run ANYTHING!

# ❌ DANGEROUS: Never do this!
# @tool  
# def read_any_file(path: str) -> str:
#     return open(path).read()  # Can read /etc/passwd!

# ✅ Right: Validate, sanitize, restrict!
print("Always validate inputs and restrict what tools can do.")
print("See our execute_python and read_file tools for safe implementations.")

---

## 🎉 Checkpoint

You've learned:
- ✅ What tools are and why agents need them
- ✅ How to create safe, well-documented tools
- ✅ Building a calculator with security in mind
- ✅ File operations with proper validation
- ✅ Code execution in a sandboxed environment
- ✅ Combining tools into a working agent

---

## ✋ Try It Yourself

Create your own custom tool! Ideas:
- A tool that gets the current date/time
- A tool that converts between units (miles to km, etc.)
- A tool that generates random data

<details>
<summary>💡 Hint: Date/Time Tool</summary>

```python
@tool
def get_datetime(format: str = "%Y-%m-%d %H:%M:%S") -> str:
    """Get the current date and time.
    
    Args:
        format: strftime format (default: YYYY-MM-DD HH:MM:SS)
    """
    from datetime import datetime
    return datetime.now().strftime(format)
```
</details>

In [ ]:
# Your custom tool here!
# @tool
# def my_custom_tool(...):
#     ...

---

## 🚀 Challenge (Optional)

Build a multi-step agent that can:
1. Search for information about a topic
2. Write Python code to analyze that information
3. Calculate statistics from the results

Example query: "Find information about DGX Spark memory, then write code to calculate how many 7B parameter models (at 2 bytes per parameter in FP16) could fit in that memory."

---

## 📖 Further Reading

- [LangChain Tools Documentation](https://python.langchain.com/docs/modules/tools/)
- [Building Custom Tools](https://python.langchain.com/docs/modules/tools/custom_tools/)
- [Tool Use Safety Best Practices](https://docs.anthropic.com/claude/docs/tool-use)

---

## 🧹 Cleanup

In [ ]:
# Comprehensive cleanup for DGX Spark
import gc

# Clear GPU memory if available
try:
    import torch
    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()
        allocated = torch.cuda.memory_allocated() / 1e9
        print(f"✅ GPU memory cleared ({allocated:.2f} GB still allocated)")
except ImportError:
    pass

# Python garbage collection
gc.collect()
print("✅ Cleanup complete!")

---

## 🎓 Summary

In this notebook, you built 5 production-ready tools:

1. **Calculator** - Safe math evaluation without `eval()`
2. **File Reader** - Sandboxed file access
3. **Web Search** - Simulated search (production: use real API)
4. **Code Executor** - Restricted Python execution
5. **API Caller** - HTTP requests with security checks

**Key Principles:**
- Always validate and sanitize inputs
- Provide clear, detailed tool descriptions
- Handle errors gracefully
- Restrict what tools can do (defense in depth)

**Next up:** Task 13.3 - LlamaIndex Query Engine with Hybrid Search